In [2]:
from chestxsim.wrappers.astra import *
from chestxsim.core import TomoGeometry

ImportError: cannot import name 'GeometricOp' from partially initialized module 'chestxsim.wrappers.base' (most likely due to a circular import) (D:\bhermosi\chestxsim-project\src\chestxsim\wrappers\base.py)

In [ ]:
geom = TomoGeometry(
    detector_size=(4288, 4288),
    pixel_size=(0.1, 0.1),
    binning_proj=8,
    SDD=1800.0,
    bucky=14.47,
    step_mm=16.077,
    nprojs=60
)


In [ ]:
print(geom.beam) 
print(geom.modality)
print(geom.DOD)

In [ ]:
from chestxsim.io import RawReader
from chestxsim.projection.steps import Projection
from chestxsim.utility import plot
from pathlib import Path 

reader = RawReader()
base_path= Path(r"D:\bhermosi\chestxsim-project\examples\volumes\CT_converted")
case_id = "NODULO_S18_S20"
att_vol = reader.read( base_path/ "mus"/ case_id)

In [ ]:
att_vol.metadata.voxel_size

In [ ]:
tomo_op= ASTRA_Tomo(geom)

In [ ]:
for attr, value in tomo_op.__dict__.items():
    print(f"- {attr}: {value}")

In [ ]:
input_vol = att_vol.volume[:,:,:,0]
vx_xyz = att_vol.metadata.voxel_size

projs = tomo_op.project(input_vol, vx_xyz)
projs.shape

reco_dim_xyz = (344, 86, 276)
reco_vx_xyz = (1.25, 5.00, 1.25)
backproj = tomo_op.backproject(projs, reco_dim_xyz, reco_vx_xyz)
backproj.shape

In [ ]:
geom.DOD

In [ ]:
plot(projs[:,:,30].T.get())
plot(backproj[:,30,:].T.get())

In [ ]:
geom_cbct = CTGeom(
    detector_size=(4288, 4288),
    pixel_size=(0.1, 0.1),
    binning_proj=8,
    SDD=1800.0,
    DOD=307.452,
    nprojs= 360,
    step_angle= 1, 
    init_angle=  -90
)
cbct = ASTRA_CBCT(geom_cbct)

In [ ]:
geom_cbct_factory = create_geometry(
    beam= BeamGeom.CONE,
    modality= Modality.CT3D, 
    detector_size=(4288, 4288),
    pixel_size=(0.1, 0.1),
    binning_proj=8,

    SDD=1800.0,
    DOD=307.452,
    nprojs= 360,
    step_angle= 1, 
    init_angle=  -90
)

In [ ]:
cbct_cpu = ASTRA_CBCT(geom_cbct_factory, Backend.CPU)

In [ ]:
cbct_cpu.hooks.backend

In [ ]:
input_vol = att_vol.volume[:,:,:,0]
vx_xyz = [att_vol.metadata.voxel_size[i]*0.85 for i in range(3)]
print(input_vol.shape)
print(vx_xyz)

In [ ]:
projs = cbct.project(input_vol, vx_xyz)
print(projs.shape)
plot(projs[:,:,90].T.get())

In [ ]:
backproj = cbct.backproject(projs, input_vol.shape, vx_xyz)
print(backproj.shape)
plot(backproj[:,:,200].T.get())

In [ ]:
# vx_size_iso = (1.0, 1.0, 1.0)
# fdk = cbct.reconstruct(
#     "FDK_CUDA",
#     projs,
#     input_vol.shape, 
#     vx_size_iso,
#     options={"FilterType": "Ram-Lak"} ) 

# plot(fdk[:,:,200].T.get()) 

In [ ]:
# vx_size_iso = (1.0, 1.0, 1.0)
# sirt = cbct.reconstruct(
#     "SIRT3D_CUDA",
#     projs,
#     input_vol.shape, 
#     vx_size_iso,
#     options={"MinConstrain": "0.0"},
#     iterations = 150) 

# plot(sirt[:,:,200].T.get()) 


In [ ]:
# vx_size_iso = (1.0, 1.0, 1.0)
# sirt = cbct.reconstruct(
#     "CGLS3D_CUDA",
#     projs,
#     input_vol.shape, 
#     vx_size_iso,
#     iterations = 100) 

# plot(sirt[:,:,200].T.get())

In [ ]:
vx_size_iso = (1.0, 1.0, 1.0)
fdk = cbct.reconstruct(
    "FDK_CUDA",
    projs,
    input_vol.shape, 
    vx_size_iso,
    options={"FilterType": "Ram-Lak"} ) 

plot(fdk[:,:,200].T.get()) 

In [ ]:
fanbeam_geom = CT2DFanGeom(
    detector_size=4288,
    pixel_size=0.1,
    binning_proj=8,
    SDD=1800.0,
    DOD=307.452,
    nprojs= 360,
    step_angle= 1, 
    init_angle=  -90
)

In [ ]:

input_img = att_vol.volume[:,:,200,0]
px_xy = (0.85, 0.85)
fanbeam = ASTRA_FANBEAMCT(fanbeam_geom)
projs = fanbeam.project(input_img, px_xy)
print(projs.shape)
plot(projs.T.get())

In [ ]:
backproj = fanbeam.backproject(projs, input_img.shape, px_xy)
fbp = fanbeam.reconstruct(
    "FBP_CUDA",
    projs,
    input_img.shape, 
    px_xy,
    options = {"FilterType": "Ram-Lak"} ) 



In [ ]:
plot(fbp.T.get())

In [ ]:
it = fanbeam.reconstruct(
    method ="SIRT_CUDA",
    sino = projs,
    reco_dim_xyz = input_img.shape, 
    reco_vx_xyz = px_xy,
    options = {'MinConstraint': 0.0},
    iterations = 1000
    )

In [ ]:
plot(it.T.get())